In [22]:
import re
import os
import csv
import urllib.request
from urllib.parse import quote
from user_agent import generate_user_agent

In [23]:
pairs=[]
with open('standList.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        pairs.append((row[0],row[1]))

In [32]:
def html_from_keyword(keyword):
    url="https://www.google.com/search?q="+keyword+"&source=lnms&tbm=isch"
    request = urllib.request.Request(url,headers={"User-Agent": generate_user_agent()})
    page = urllib.request.urlopen(request)
    return str(page.read())

def get_image_link(keyword):
    page_content = html_from_keyword(keyword)
    link_list = re.findall('"ou":"(.*?)"', page_content)
    return set(link_list)

def download(keyword,img_dir,prefix="",img_per_search=1):
    if(len(prefix)>0):
        prefix+="+"
        prefix=prefix.replace(' ','+')
    modified_keyword=prefix+keyword.replace(' ','+')
    image_links=set()
    attempt_count=0
    while(len(image_links)==0):
        if(attempt_count==5):
            break
        image_links = image_links.union(get_image_link(modified_keyword))
    print(keyword,"has",len(image_links),"image links")
    print("The link is","https://www.google.com/search?q="+modified_keyword+"&source=lnms&tbm=isch")
    count=0
    for image_link in image_links:
        request = urllib.request.Request(image_link, headers = {"User-Agent": generate_user_agent()})
        count2=1
        moveOn=False
        while(True):
            try:
                response = urllib.request.urlopen(request)
                break
            except:
                if(count2==5):
                    print("Request failed 5 times. Moving on to next link")
                    moveOn=True
                    break
                count2+=1
                print("Error upon request, requesting for the",count2,"time.")
        if moveOn:
            continue
        data = response.read()
        file_path = img_dir+keyword+str(count)+".png"
        with open(file_path,'wb') as wf:
            wf.write(data)
            print("Image saved as",file_path)
        count+=1
        if(count==img_per_search):
            break

In [34]:
path = os.getcwd()+'\images\\'
if not os.path.exists(path):
    os.mkdir(path)
img_per_search=5
prefix="JoJo's Bizarre Adventure"
for pair in pairs:
    name=pair[0]
    stand=pair[1]
    print(name,stand)
    new_path=path+name+"_"+stand+"\\"
    if not os.path.exists(new_path):
        os.mkdir(new_path)
    download(name,new_path,prefix,img_per_search)
    print("Breaking")
    download(stand,new_path,prefix+" Stand",img_per_search)
print("Done!")

Jotaro Kujo Star Platinum
Jotaro Kujo has 100 image links
The link is https://www.google.com/search?q=JoJo's+Bizarre+Adventure+Jotaro+Kujo&source=lnms&tbm=isch
Image saved as C:\Users\65853\Desktop\CS\Project\images\Jotaro Kujo_Star Platinum\Jotaro Kujo0.png
Image saved as C:\Users\65853\Desktop\CS\Project\images\Jotaro Kujo_Star Platinum\Jotaro Kujo1.png
Image saved as C:\Users\65853\Desktop\CS\Project\images\Jotaro Kujo_Star Platinum\Jotaro Kujo2.png
Image saved as C:\Users\65853\Desktop\CS\Project\images\Jotaro Kujo_Star Platinum\Jotaro Kujo3.png
Image saved as C:\Users\65853\Desktop\CS\Project\images\Jotaro Kujo_Star Platinum\Jotaro Kujo4.png
Breaking
Star Platinum has 100 image links
The link is https://www.google.com/search?q=JoJo's+Bizarre+Adventure+Stand+Star+Platinum&source=lnms&tbm=isch
Image saved as C:\Users\65853\Desktop\CS\Project\images\Jotaro Kujo_Star Platinum\Star Platinum0.png
Image saved as C:\Users\65853\Desktop\CS\Project\images\Jotaro Kujo_Star Platinum\Star Plat